## Import Statements

In [1]:
# Import sciPy as a statistical package
from scipy import stats

# Import pickle to read the data
import pickle
# Import pandas for data managment
import pandas as pd
# Import numpy for supportive calculations
import numpy as np

# Import the matplotlib python package for plotting
import matplotlib.pyplot as plt
# Set the plot display format to 'retina' for more detailed graphs
%config InlineBackend.figure_format='retina'
%matplotlib inline

from matplotlib import colors

from sklearn.model_selection import train_test_split


## Data Cleaning

In [2]:
# Open the pickle file
with open('insider_data.pkl', 'rb') as f:
    # Workaround to read pickle file to do bug in Pandas 1.3.0
    # (The bug is fixed in version 1.3.1)
    data = pd.read_pickle(f)
    
# Define the value of the trade ($ value)
data['value_of_trade'] = data['shares_transacted']*data['executionPrice']


In [3]:
data.sector.unique()

array(['Energy', 'Consumer Staples', 'Utilities', 'Materials',
       'Financials', 'Health Care', 'Consumer Discretionary',
       'Information Technology', 'Industrials', 'Communication Services',
       None, 'Real Estate'], dtype=object)

In [4]:
data

companyid   tic  ownerObjectId  insiderTradeId  \
datadate   issueid                                                    
2003-12-31 00851201     296104  PETD         615007         3002586   
           03065101     341990   DAR        9993943         1944208   
2004-01-05 00797401     292092    NI         753642         3311983   
           00869201     297620   PCH         533349         3337544   
2004-01-06 00797401     292092    NI         753642          741180   
...                        ...   ...            ...             ...   
2021-07-02 17710901   13413134   CAI      204975497         7291252   
           17710901   13413134   CAI      204975497         7291253   
           17801501   34125194  LULU      412691818         7293539   
           18336601   13103664     H      112268879         7291440   
           18336601   13103664     H      276199276         7292182   

                                       transaction_type  shares_transacted  \
datadate   issueid                                                           
2003-12-31 00851201             Open Market Disposition            -7899.0   
           03065101             Open Market Acquisition           185000.0   
2004-01-05 00797401             Open Market Disposition           -54358.0   
           00869201  Open Market Derivative Acquisition              148.6   
2004-01-06 00797401             Open Market Disposition            -4063.0   
...                                                 ...                ...   
2021-07-02 17710901             Open Market Disposition            -1947.0   
           17710901             Open Market Disposition            -2937.0   
           17801501             Open Market Disposition            -4000.0   
           18336601             Open Market Disposition           -93750.0   
           18336601             Open Market Disposition           -93750.0   

                     executionPrice  percentOfSharesTraded  \
datadate   issueid                                           
2003-12-31 00851201       24.645948              -0.050542   
           03065101        2.694643               0.292230   
2004-01-05 00797401       21.765000              -0.020710   
           00869201        0.000000                    NaN   
2004-01-06 00797401       21.765000              -0.001548   
...                             ...                    ...   
2021-07-02 17710901       25.900000                    NaN   
           17710901       25.899475                    NaN   
           17801501      366.755068              -0.003201   
           18336601       77.630000              -0.227822   
           18336601       77.630000              -0.227822   

                     pct_change_in_shares_held_by_insider  \
datadate   issueid                                          
2003-12-31 00851201                            -28.312842   
           03065101                              0.890176   
2004-01-05 00797401                            -11.179069   
           00869201                              0.896088   
2004-01-06 00797401                             -0.912238   
...                                                   ...   
2021-07-02 17710901                                   NaN   
           17710901                                   NaN   
           17801501                            -65.445026   
           18336601                            -17.774836   
           18336601                            -17.774836   

                     n_shares_owned_post_transaction  ...       yield  \
datadate   issueid                                    ...               
2003-12-31 00851201                          20000.0  ...   YIELD_REG   
           03065101                       20967387.0  ...   YIELD_REG   
2004-01-05 00797401                         431890.0  ...  YIELD_HIGH   
           00869201                          16732.0  ...  YIELD_HIGH   
2004-01-06 00797401                        

In [5]:
# Make a copy of the data for testing
df = data.copy()

In [6]:
# Add one-hot encoding for sectors
sector_values = list(data.sector.unique())
sector_values.remove(None)

df[sector_values] = pd.get_dummies(data['sector'])

In [7]:
# Add one-hot encoding for transaction_type
transaction_values = list(data.transaction_type.unique())

df[transaction_values] = pd.get_dummies(data['transaction_type'])

In [8]:
# Add one-hot encoding for cohort
cohort_names = list(df.cohort_name.unique())
[cohort_names.remove(x) for x in cohort_names if not type(x) == str]

df[cohort_names] = pd.get_dummies(df['cohort_name'])

In [9]:
df

companyid   tic  ownerObjectId  insiderTradeId  \
datadate   issueid                                                    
2003-12-31 00851201     296104  PETD         615007         3002586   
           03065101     341990   DAR        9993943         1944208   
2004-01-05 00797401     292092    NI         753642         3311983   
           00869201     297620   PCH         533349         3337544   
2004-01-06 00797401     292092    NI         753642          741180   
...                        ...   ...            ...             ...   
2021-07-02 17710901   13413134   CAI      204975497         7291252   
           17710901   13413134   CAI      204975497         7291253   
           17801501   34125194  LULU      412691818         7293539   
           18336601   13103664     H      112268879         7291440   
           18336601   13103664     H      276199276         7292182   

                                       transaction_type  shares_transacted  \
datadate   issueid                                                           
2003-12-31 00851201             Open Market Disposition            -7899.0   
           03065101             Open Market Acquisition           185000.0   
2004-01-05 00797401             Open Market Disposition           -54358.0   
           00869201  Open Market Derivative Acquisition              148.6   
2004-01-06 00797401             Open Market Disposition            -4063.0   
...                                                 ...                ...   
2021-07-02 17710901             Open Market Disposition            -1947.0   
           17710901             Open Market Disposition            -2937.0   
           17801501             Open Market Disposition            -4000.0   
           18336601             Open Market Disposition           -93750.0   
           18336601             Open Market Disposition           -93750.0   

                     executionPrice  percentOfSharesTraded  \
datadate   issueid                                           
2003-12-31 00851201       24.645948              -0.050542   
           03065101        2.694643               0.292230   
2004-01-05 00797401       21.765000              -0.020710   
           00869201        0.000000                    NaN   
2004-01-06 00797401       21.765000              -0.001548   
...                             ...                    ...   
2021-07-02 17710901       25.900000                    NaN   
           17710901       25.899475                    NaN   
           17801501      366.755068              -0.003201   
           18336601       77.630000              -0.227822   
           18336601       77.630000              -0.227822   

                     pct_change_in_shares_held_by_insider  \
datadate   issueid                                          
2003-12-31 00851201                            -28.312842   
           03065101                              0.890176   
2004-01-05 00797401                            -11.179069   
           00869201                              0.896088   
2004-01-06 00797401                             -0.912238   
...                                                   ...   
2021-07-02 17710901                                   NaN   
           17710901                                   NaN   
           17801501                            -65.445026   
           18336601                            -17.774836   
           18336601                            -17.774836   

                     n_shares_owned_post_transaction  ...  financials  tmt  \
datadate   issueid                                    ...                    
2003-12-31 00851201                          20000.0  ...           0    0   
           03065101                       20967387.0  ...           0    0   
2004-01-05 00797401                         431890.0  ...           0    0   
           00869201                          16732.0  ...           0    0   
2004-01-06 00

In [10]:
# Define the list of variables
variables = ['percentOfSharesTraded', 'pct_change_in_shares_held_by_insider',
            'n_shares_owned_post_transaction', 'beta_tv_median', 'marketcap'] + \
            sector_values + transaction_values + cohort_names

In [11]:
objective_var = 'fwd_21d_return'

In [12]:
# Drop na values
df.dropna(subset=variables+[objective_var], how='any', inplace=True)

# Display the length of the data
len(df)

409658

In [13]:
def sign_classifier(percent):
    
    # Classify the forward 21 day return for each trade as positive or negative
    # Returns: 0 for negative and 1 for positive percent increase in stock
    
    if percent >= 0:
        return 1
    else:
        return 0

In [14]:
df[objective_var+'_classified'] = df[objective_var].apply(sign_classifier)
df['fwd_21d_return_classified'].value_counts()

0    207527
1    202131
Name: fwd_21d_return_classified, dtype: int64

In [15]:
def sample_df_equally_by_group(df, column, n):
    # your answer goes here
    df0 = df[df[column]==0].sample(n=n)
    df1 = df[df[column]==1].sample(n=n)
    df2 = [df0, df1]
    final = pd.concat(df2)
    
    return final

In [16]:
final_data = sample_df_equally_by_group(df, column="fwd_21d_return_classified", n=100000)
final_data.shape

(200000, 84)

In [17]:
# Define the y
y = final_data['fwd_21d_return_classified']
y

datadate    issueid 
2015-11-09  00853001    0
2015-12-17  17913201    0
2004-11-03  00225501    0
2014-03-17  03047301    0
2013-12-13  16392001    0
                       ..
2008-07-16  01695501    1
2012-01-26  02380901    1
2007-12-12  00579201    1
2007-03-15  15685701    1
2011-05-17  14391001    1
Name: fwd_21d_return_classified, Length: 200000, dtype: int64

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier

estimators = {
    'Logistic Regression': LogisticRegression(),
    'k-Nearest Neighbor': KNeighborsClassifier(),
    'MLP Classifier': MLPClassifier(),
    'Decision Tree': DecisionTreeClassifier(criterion="entropy")
    }

In [19]:
def classifiers_test(features, target, estimators, variables):
    
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score
    
    X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=3000)
    
    for estimator_name, estimator_object, in estimators.items():
        
        model = estimator_object.fit(X_train[variables], y_train)
        
        yhat = model.predict(X_test[variables])
        acc = accuracy_score(y_test, yhat)
        
        print(estimator_name + '\n\t' + f'Classification accuracy on the test data: {acc:.2%}' + '\n')
        
        long_or_short = pd.Series(yhat).apply(lambda x: 1.0 if x == 1 else -1.0)
                        
        returns = long_or_short*X_test['fwd_21d_return'].values
        
        print('\t\tMean Return: {:.4f}, Standard Deviation: {:.4f}'.format(np.mean(returns), np.std(returns)))
        
        if estimator_name == 'Decision Tree':
            return long_or_short, X_test['fwd_21d_return']
        

In [20]:
trade, returns = classifiers_test(final_data, y, estimators, variables)

Logistic Regression
	Classification accuracy on the test data: 50.22%

		Mean Return: -0.0052, Standard Deviation: 0.1165


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


k-Nearest Neighbor
	Classification accuracy on the test data: 53.20%

		Mean Return: 0.0047, Standard Deviation: 0.1166
MLP Classifier
	Classification accuracy on the test data: 49.66%

		Mean Return: 0.0051, Standard Deviation: 0.1165
Decision Tree
	Classification accuracy on the test data: 60.05%

		Mean Return: 0.0136, Standard Deviation: 0.1159
